In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
import random
from gensim.models import KeyedVectors
import keras
from sklearn.model_selection import KFold
from keras.preprocessing.text import Tokenizer
from keras.initializers import Constant
from sklearn.metrics import precision_recall_fscore_support

In [4]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-08-21 10:28:30--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.4.14
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.4.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  3.49MB/s    in 7m 35s  

2020-08-21 10:36:05 (3.45 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [5]:
!gunzip GoogleNews-vectors-negative300.bin.gz

In [7]:
w2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [8]:
w2vec_model.vectors.shape

(3000000, 300)

In [9]:
all_data = pd.read_csv('all_sentences.csv', encoding='utf-8', index_col=False)
crowd_sourced = pd.read_csv('crowdsourced.csv', encoding='utf-8', index_col=False)
groundtruth = pd.read_csv('groundtruth.csv', encoding='utf-8', index_col=False)

In [13]:
len(all_data), len(crowd_sourced), len(groundtruth)

(32072, 22501, 1032)

In [14]:
data = pd.concat([crowd_sourced, groundtruth])

In [15]:
data.shape

(23533, 10)

In [16]:
data = data.sample(frac=1).reset_index(drop=True)

In [17]:
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [18]:
input_sentences = [text_to_wordlist(text).split() for text in data["Text"].values.tolist()]
labels = data["Verdict"].values.tolist()

In [21]:
len(input_sentences)

23533

In [22]:
# Initialize word2id and label2id dictionaries that will be used to encode words and labels
word2id = dict()
label2id = dict()

max_words = 0 # maximum number of words in a sentence

# Construction of word2id dict
for sentence in input_sentences:
    for word in sentence:
        # Add words to word2id dict if not exist
        if word.lower() not in word2id:
            word2id[word.lower()] = len(word2id)
    # If length of the sentence is greater than max_words, update max_words
    if len(sentence) > max_words:
        max_words = len(sentence)
    
# Construction of label2id and id2label dicts
label2id = {l: i for i, l in enumerate(set(labels))}
id2label = {v: k for k, v in label2id.items()}
id2label

{0: 0, 1: 1, 2: -1}

In [23]:
max_words

163

In [25]:
# Encode input words and labels
X = [[word2id[word.lower()] for word in sentence] for sentence in input_sentences]
Y = [label2id[label] for label in labels]

# Apply Padding to X
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, max_words)

# Convert Y to numpy array
Y = keras.utils.to_categorical(Y, num_classes=len(label2id), dtype='float32')

# Print shapes
print("Shape of X: {}".format(X.shape))
print("Shape of Y: {}".format(Y.shape))

Shape of X: (23533, 163)
Shape of Y: (23533, 3)


In [29]:
len(word2id)

11954

In [55]:
nb_words = len(word2id)+1

embedding_matrix = np.zeros((nb_words, 300))
for word, i in word2id.items():
    if word in w2vec_model.vocab:
        embedding_matrix[i] = w2vec_model.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 1167


In [63]:
def buildmodel():
    embedding_dim = 300 # The dimension of word embeddings

    # Define input tensor
    sequence_input = keras.Input(shape=(max_words,), dtype='int32')

    # Word embedding layer
    embedded_inputs =keras.layers.Embedding(len(word2id) + 1,
                                            embedding_dim,
                                            embeddings_initializer=Constant(embedding_matrix),
                                            input_length=max_words)(sequence_input)

    # Apply dropout to prevent overfitting
    embedded_inputs = keras.layers.Dropout(0.2)(embedded_inputs)

    # Apply Bidirectional LSTM over embedded inputs
    lstm_outs = keras.layers.wrappers.Bidirectional(
        keras.layers.LSTM(embedding_dim, return_sequences=True)
    )(embedded_inputs)

    # Apply dropout to LSTM outputs to prevent overfitting
    lstm_outs = keras.layers.Dropout(0.2)(lstm_outs)

    # Attention Mechanism - Generate attention vectors
    input_dim = int(lstm_outs.shape[2])
    permuted_inputs = keras.layers.Permute((2, 1))(lstm_outs)
    attention_vector = keras.layers.TimeDistributed(keras.layers.Dense(1))(lstm_outs)
    attention_vector = keras.layers.Reshape((max_words,))(attention_vector)
    attention_vector = keras.layers.Activation('softmax', name='attention_vec')(attention_vector)
    attention_output = keras.layers.Dot(axes=1)([lstm_outs, attention_vector])

    # Last layer: fully connected with softmax activation
    fc = keras.layers.Dense(embedding_dim, activation='relu')(attention_output)
    output = keras.layers.Dense(len(label2id), activation='softmax')(fc)

    # Finally building model
    model = keras.Model(inputs=[sequence_input], outputs=output)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam')

    # Print model summary
#     print(model.summary())
    return model

In [57]:
from sklearn.model_selection import KFold

In [58]:
num_folds = 5

In [59]:
kfold = KFold(n_splits=num_folds)

In [60]:
macro_precision_per_fold = []
macro_recall_per_fold = []
macro_f1_per_fold = []
micro_precision_per_fold = []
micro_recall_per_fold = []
micro_f1_per_fold = []

In [64]:
fold_no = 1
for train, test in kfold.split(X, Y):
    model = buildmodel()
    model.fit(X[train], Y[train], epochs=10, batch_size=64, verbose=0)
    pred = model.predict(X[test])
    y_pred = pred.argmax(axis=-1)
    Y_test = Y[test].argmax(axis=-1)
    scores = precision_recall_fscore_support(Y_test, y_pred, average='macro')
    macro_precision_per_fold.append(scores[0])
    macro_recall_per_fold.append(scores[1])
    macro_f1_per_fold.append(scores[2])
    scores = precision_recall_fscore_support(Y_test, y_pred, average='micro')
    micro_precision_per_fold.append(scores[0])
    micro_recall_per_fold.append(scores[1])
    micro_f1_per_fold.append(scores[2])

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [61]:
sum(macro_precision_per_fold)/len(macro_precision_per_fold)

0.6236142556147392

In [62]:
sum(macro_recall_per_fold)/len(macro_recall_per_fold)

0.6112308030793936

In [63]:
sum(micro_precision_per_fold)/len(micro_precision_per_fold)

0.7480980258263885

In [64]:
sum(micro_recall_per_fold)/len(micro_recall_per_fold)

0.7480980258263885